In [ ]:
import warnings

# import aesara.tensor as at
import pytensor.tensor as pt
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import scipy as sp

from scipy.special import expit  # sigmoid
from scipy import stats
from scipy.special import expit as logistic
from scipy.special import softmax
from scipy.special import logit

%config InlineBackend.figure_format = 'retina'
warnings.simplefilter(action="ignore", category=FutureWarning)
RANDOM_SEED = 8927
np.random.seed(286)

In [ ]:
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89


def standardize(series):
    """Standardize a pandas series"""
    return (series - series.mean()) / series.std()

#### Code 11.1

In [ ]:
d = pd.read_csv("Data/chimpanzees.csv", sep=";")
# we change "actor" to zero-index
d.actor = d.actor - 1
d

#### Code 11.2

In [ ]:
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

#### Code 11.3

In [ ]:
d.groupby("treatment").first()[["prosoc_left", "condition"]]

#### Code 11.4 and 11.5

In [ ]:
with pm.Model() as m11_1:
    a = pm.Normal("a", 0.0, 10.0)
    p = pm.Deterministic("p", pm.math.invlogit(a))
    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    prior_11_1 = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
idata_11_1 = prior_11_1["prior"]

# need it to recreate the prior pred plot:
with pm.Model() as m11_1bis:
    a = pm.Normal("a", 0.0, 1.5)
    p = pm.Deterministic("p", pm.math.invlogit(a))
    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    prior_11_1bis = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
idata_11_1bis = prior_11_1bis["prior"]

#### Code 11.6

In [ ]:
ax = az.plot_density(
    [idata_11_1, idata_11_1bis],
    data_labels=["a ~ Normal(0, 10)", "a ~ Normal(0, 1.5)"],
    group="prior",
    colors=["k", "b"],
    var_names=["p"],
    point_estimate=None,
)[0]
ax[0].set_xlabel("prior prob pull left")
ax[0].set_ylabel("Density")
ax[0].set_title("Prior predictive simulations for p");

#### Code 11.7 - 11.9

In [ ]:
with pm.Model() as m11_2:
    a = pm.Normal("a", 0.0, 1.5)
    b = pm.Normal("b", 0.0, 10.0, shape=4)

    p = pm.math.invlogit(a + b[d.treatment.values])
    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    prior_11_2 = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
prior_2 = prior_11_2["prior"]

with pm.Model() as m11_3:
    a = pm.Normal("a", 0.0, 1.5)
    b = pm.Normal("b", 0.0, 0.5, shape=4)

    p = pm.math.invlogit(a + b[d.treatment.values])
    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    prior_11_3 = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
prior_3 = prior_11_3["prior"]

In [ ]:
p_treat1, p_treat2 = (
    logistic(prior_2["a"] + prior_2["b"].sel(b_dim_0=0)),
    logistic(prior_2["a"] + prior_2["b"].sel(b_dim_0=1)),
)
p_treat1_bis, p_treat2_bis = (
    logistic(prior_3["a"] + prior_3["b"].sel(b_dim_0=0)),
    logistic(prior_3["a"] + prior_3["b"].sel(b_dim_0=1)),
)

ax = az.plot_density(
    [np.abs(p_treat1 - p_treat2).values, np.abs(p_treat1_bis - p_treat2_bis).values],
    data_labels=["b ~ Normal(0, 10)", "b ~ Normal(0, 0.5)"],
    group="prior",
    colors=["k", "b"],
    point_estimate=None,
)[0]
ax[0].set_xlabel("prior diff between treatments")
ax[0].set_ylabel("Density")
ax[0].set_title(None);

In [ ]:
np.abs(p_treat1_bis - p_treat2_bis).mean().values

In [ ]:
# p_treat1_bis

#### Code 11.10

In [ ]:
actor_idx, actors = pd.factorize(d.actor)
treat_idx, treatments = pd.factorize(d.treatment)

#### Code 11.11

In [ ]:
with pm.Model() as m11_4:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    actor_id = pm.intX(pm.Data("actor_id", actor_idx))
    treat_id = pm.intX(pm.Data("treat_id", treat_idx))
    p = pm.Deterministic("p", pm.math.invlogit(a[actor_id] + b[treat_id]))

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

az.summary(trace_11_4, var_names=["a", "b"], round_to=2)

#### Code 11.12

In [ ]:
az.plot_forest(trace_11_4, var_names=["a"], transform=logistic, combined=True);

#### Code 11.13

In [ ]:
ax = az.plot_forest(trace_11_4, var_names=["b"], combined=True)
ax[0].set_yticklabels(["L/P", "R/P", "L/N", "R/N"]);

#### Code 11.14

In [ ]:
db13 = trace_11_4.posterior["b"].sel(b_dim_0=0) - trace_11_4.posterior["b"].sel(b_dim_0=2)
db24 = trace_11_4.posterior["b"].sel(b_dim_0=1) - trace_11_4.posterior["b"].sel(b_dim_0=3)
az.plot_forest([db13.values, db24.values], model_names=["db13", "db24"], combined=True);

#### Code 11.15

In [ ]:
pl = d.groupby(["actor", "treatment"]).agg("mean")["pulled_left"].unstack()
pl

#### Code 11.16 and 11.17

In [ ]:
with m11_4:
    pm.set_data({"actor_id": np.repeat(range(7), 4), "treat_id": list(range(4)) * 7})
    ppd = pm.sample_posterior_predictive(trace_11_4, random_seed=RANDOM_SEED, var_names=["p"])[
        "posterior_predictive"
    ]["p"]
p_mu = np.array(ppd.mean(["chain", "draw"])).reshape((7, 4))

In [ ]:
pl_val = pl.stack().values
_, (ax0, ax1) = plt.subplots(2, 1, figsize=(12, 6))
alpha, xoff, yoff = 0.6, 0.3, 0.05

ax0.plot([7 * 4 - 0.5] * 2, [0, 1], c="k", alpha=0.4, lw=1)
ax0.axhline(0.5, ls="--", c="k", alpha=0.4)
for actor in range(len(actors)):
    ax0.plot(
        [actor * 4, actor * 4 + 2],
        [pl.loc[actor, 0], pl.loc[actor, 2]],
        "-",
        c="b",
        alpha=alpha,
    )
    ax0.plot(
        [actor * 4 + 1, actor * 4 + 3],
        [pl.loc[actor, 1], pl.loc[actor, 3]],
        "-",
        c="b",
        alpha=alpha,
    )
    ax0.plot(
        [actor * 4, actor * 4 + 1],
        [pl.loc[actor, 0], pl.loc[actor, 1]],
        "o",
        c="b",
        fillstyle="none",
        ms=6,
        alpha=alpha,
    )
    ax0.plot(
        [actor * 4 + 2, actor * 4 + 3],
        [pl.loc[actor, 2], pl.loc[actor, 3]],
        "o",
        c="b",
        ms=6,
        alpha=alpha,
    )
    ax0.plot([actor * 4 - 0.5] * 2, [0, 1], c="k", alpha=0.4, lw=1)
    ax0.text(actor * 4 + 0.5, 1.1, f"actor {actor + 1}", fontsize=12)
    if actor == 0:
        ax0.text(actor * 4 - xoff, pl.loc[actor, 0] - 2 * yoff, "R/N")
        ax0.text(actor * 4 + 1 - xoff, pl.loc[actor, 1] + yoff, "L/N")
        ax0.text(actor * 4 + 2 - xoff, pl.loc[actor, 2] - 2 * yoff, "R/P")
        ax0.text(actor * 4 + 3 - xoff, pl.loc[actor, 3] + yoff, "L/P")
ax0.set_xticks([])
ax0.set_ylabel("proportion left lever", labelpad=10)
ax0.set_title("observed proportions", pad=25)

ax1.plot([range(28), range(28)], az.hdi(ppd)["p"].T, "k-", lw=2, alpha=alpha)
ax1.plot([7 * 4 - 0.5] * 2, [0, 1], c="k", alpha=0.4, lw=1)
ax1.axhline(0.5, ls="--", c="k", alpha=0.4)
for actor in range(len(actors)):
    ax1.plot(
        [actor * 4, actor * 4 + 2],
        [p_mu[actor, 0], p_mu[actor, 2]],
        "-",
        c="k",
        alpha=alpha,
    )
    ax1.plot(
        [actor * 4 + 1, actor * 4 + 3],
        [p_mu[actor, 1], p_mu[actor, 3]],
        "-",
        c="k",
        alpha=alpha,
    )
    ax1.plot(
        [actor * 4, actor * 4 + 1],
        [p_mu[actor, 0], p_mu[actor, 1]],
        "o",
        c="k",
        fillstyle="none",
        ms=6,
        alpha=alpha,
    )
    ax1.plot(
        [actor * 4 + 2, actor * 4 + 3],
        [p_mu[actor, 2], p_mu[actor, 3]],
        "o",
        c="k",
        ms=6,
        alpha=alpha,
    )
    ax1.plot([actor * 4 - 0.5] * 2, [0, 1], c="k", alpha=0.4, lw=1)
    ax1.text(actor * 4 + 0.5, 1.1, f"actor {actor + 1}", fontsize=12)
ax1.set_xticks([])
ax1.set_ylabel("proportion left lever", labelpad=10)
ax1.set_title("posterior predictions", pad=25)

#### Code 11.18

In [ ]:
side = d.prosoc_left.values  # right 0, left 1
cond = d.condition.values  # no partner 0, partner 1

#### Code 11.19

In [ ]:
with pm.Model() as m11_5:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    bs = pm.Normal("bs", 0.0, 0.5, shape=2)
    bc = pm.Normal("bc", 0.0, 0.5, shape=2)

    p = pm.math.invlogit(a[actor_idx] + bs[side] + bc[cond])

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_5 = pm.sample(random_seed=RANDOM_SEED)

#### Code 11.20
As we changed the data of `m11_4` above, we need to sample from it again, with the original data:

In [ ]:
with m11_4:
    pm.set_data({"actor_id": actor_idx, "treat_id": treat_idx})
    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

az.compare({"m11_4": trace_11_4, "m11_5": trace_11_5})

#### Code 11.23

In [ ]:
post_b_11_4 = az.extract_dataset(trace_11_4["posterior"])["b"]
np.exp(np.array(post_b_11_4[3, :]) - np.array(post_b_11_4[1, :])).mean().round(3)

#### Code 11.24

In [ ]:
d = pd.read_csv("Data/chimpanzees.csv", sep=";")
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition

d_aggregated = (
    d.groupby(["treatment", "actor"]).sum().reset_index()[["treatment", "actor", "pulled_left"]]
)
d_aggregated.head(10)

#### Code 11.25

In [ ]:
with pm.Model() as m11_6:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    p = pm.Deterministic(
        "p", pm.math.invlogit(a[d_aggregated.actor.values] + b[d_aggregated.treatment.values])
    )

    pulled_left = pm.Binomial("pulled_left", 18, p, observed=d_aggregated.pulled_left)

    trace_11_6 = pm.sample(random_seed=RANDOM_SEED)

#### Code 11.26
ArviZ won't even let you compare models with different observations:

In [ ]:
az.compare({"m11_4": trace_11_4, "m11_6": trace_11_6})

#### Code 11.27

In [ ]:
# deviance of aggregated 6-in-9
(-2 * stats.binom.logpmf(6, 9, 0.2)).round(5)

In [ ]:
# deviance of dis-aggregated
-2 * stats.bernoulli.logpmf([1, 1, 1, 1, 1, 1, 0, 0, 0], 0.2).sum().round(5)

#### Code 11.28

In [ ]:
d_ad = pd.read_csv("Data/UCBadmit.csv", sep=";")
d_ad

#### Code 11.29

In [ ]:
gid = (d_ad["applicant.gender"] == "female").astype(int).values

with pm.Model() as m11_7:
    a = pm.Normal("a", 0, 1.5, shape=2)
    p = pm.Deterministic("p", pm.math.invlogit(a[gid]))

    admit = pm.Binomial("admit", p=p, n=d_ad.applications.values, observed=d_ad.admit.values)

    trace_11_7 = pm.sample(random_seed=RANDOM_SEED)
az.summary(trace_11_7, var_names=["a"], round_to=2)

#### Code 11.30

In [ ]:
post_a_11_7 = az.extract_dataset(trace_11_7["posterior"])["a"]
diff_a = post_a_11_7[0, :] - post_a_11_7[1, :]
diff_p = logistic(post_a_11_7[0, :]) - logistic(post_a_11_7[1, :])
az.summary({"diff_a": diff_a, "diff_p": diff_p}, kind="stats", round_to=2)

#### Code 11.31

In [ ]:
with m11_7:
    ppc = pm.sample_posterior_predictive(
        trace_11_7, random_seed=RANDOM_SEED, var_names=["admit", "p"]
    )
    pp_p = ppc["posterior_predictive"]["p"]
    pp_admit = ppc["posterior_predictive"]["admit"] / d_ad.applications.values[None, :]


p_mu = np.array(pp_p.mean(["chain", "draw"]))
p_std = np.array(pp_p.std(["chain", "draw"]))
admit_mu = np.array(pp_admit.mean(["chain", "draw"]))
admit_std = np.array(pp_admit.std(["chain", "draw"]))

In [ ]:
for i in range(6):
    x = 1 + 2 * i

    y1 = d_ad.admit[x] / d_ad.applications[x]
    y2 = d_ad.admit[x + 1] / d_ad.applications[x + 1]

    plt.plot([x, x + 1], [y1, y2], "-C0o", alpha=0.6, lw=2)
    plt.text(x + 0.25, (y1 + y2) / 2 + 0.05, d_ad.dept[x])

plt.plot(range(1, 13), p_mu, "ko", fillstyle="none", ms=6, alpha=0.6)
plt.plot([range(1, 13), range(1, 13)], az.hdi(trace_11_7)["p"].T, "k-", lw=1, alpha=0.6)
plt.plot([range(1, 13), range(1, 13)], az.hdi(pp_admit)["admit"].T, "k+", ms=6, alpha=0.6)

plt.xlabel("case")
plt.ylabel("admit")
plt.title("Posterior validation check")
plt.ylim(0, 1);

#### Code 11.32

In [ ]:
dept_id = pd.Categorical(d_ad["dept"]).codes

with pm.Model() as m11_8:
    a = pm.Normal("a", 0, 1.5, shape=2)
    delta = pm.Normal("delta", 0, 1.5, shape=6)

    p = pm.math.invlogit(a[gid] + delta[dept_id])

    admit = pm.Binomial("admit", p=p, n=d_ad.applications.values, observed=d_ad.admit.values)

    trace_11_8 = pm.sample(2000, random_seed=RANDOM_SEED)
az.summary(trace_11_8, round_to=2)

#### Code 11.33

In [ ]:
post_a_11_8 = az.extract_dataset(trace_11_8["posterior"])["a"]
diff_a = post_a_11_8[0, :] - post_a_11_8[1, :]
diff_p = logistic(post_a_11_8[0, :]) - logistic(post_a_11_8[1, :])
az.summary({"diff_a": diff_a, "diff_p": diff_p}, kind="stats", round_to=2)

#### Code 11.34

In [ ]:
pg = pd.DataFrame(index=["male", "female"], columns=d_ad.dept.unique())
for dep in pg.columns:
    pg[dep] = (
        d_ad.loc[d_ad.dept == dep, "applications"]
        / d_ad.loc[d_ad.dept == dep, "applications"].sum()
    ).values
pg.round(2)

#### Code 11.35

In [ ]:
y = np.random.binomial(n=1000, p=1 / 1000, size=10_000)
y.mean(), y.var()

#### Code 11.36

In [ ]:
dk = pd.read_csv("Data/Kline", sep=";")
dk

#### Code 11.37

In [ ]:
P = standardize(np.log(dk.population)).values
c_id = (dk.contact == "high").astype(int).values

#### Code 11.38

In [ ]:
ax = az.plot_kde(
    np.random.lognormal(0.0, 10.0, size=10_000),
    label="a ~ LogNormal(0, 10)",
    plot_kwargs={"color": "k"},
)
ax.set_xlabel("mean number of tools")
ax.set_ylabel("Density")
ax.set_title("");

#### Code 11.39

In [ ]:
a = np.random.normal(0.0, 10.0, size=10_000)
np.exp(a).mean()

#### Code 11.40

In [ ]:
ax = az.plot_kde(np.random.lognormal(3.0, 0.5, size=20_000), label="a ~ LogNormal(3, 0.5)")
ax.set_xlabel("mean number of tools")
ax.set_ylabel("Density")
ax.set_title("");

#### Code 11.41 to 11.44

In [ ]:
def kline_prior_plot(N: int = 100, b_prior: str = "bespoke", x_scale: str = "stdz", ax=None):
    """
    Utility function to plot prior predictive checks for Kline Poisson model.
    N: number of prior predictive trends.

    """
    if ax is None:
        _, ax = plt.subplots()
    ax.set_ylabel("total tools")

    itcpts = np.random.normal(3.0, 0.5, N)
    if b_prior == "conventional":
        slopes = np.random.normal(0.0, 10.0, N)
        ax.set_title("b ~ Normal(0, 10)")
    elif b_prior == "bespoke":
        slopes = np.random.normal(0.0, 0.2, N)
        ax.set_title("b ~ Normal(0, 0.2)")
    else:
        raise ValueError(
            "Prior for slopes (b_prior) can only be either 'conventional' or 'bespoke'."
        )

    x_seq = np.linspace(np.log(100), np.log(200_000), N)
    ax.set_ylim((0, 500))
    if x_scale == "log":
        for a, b in zip(itcpts, slopes):
            ax.plot(x_seq, np.exp(a + b * x_seq), "k", alpha=0.4)
        ax.set_xlabel("log population")
    elif x_scale == "natural":
        for a, b in zip(itcpts, slopes):
            ax.plot(np.exp(x_seq), np.exp(a + b * x_seq), "k", alpha=0.4)
        ax.set_xlabel("population")
    else:
        x_seq = np.linspace(-2, 2, N)
        for a, b in zip(itcpts, slopes):
            ax.plot(x_seq, np.exp(a + b * x_seq), "k", alpha=0.4)
        ax.set_ylim((0, 100))
        ax.set_xlabel("log population (std)")

    return ax

In [ ]:
_, ax = plt.subplots(2, 2, figsize=(10, 10))
kline_prior_plot(b_prior="conventional", x_scale="stdz", ax=ax[0][0])
kline_prior_plot(b_prior="bespoke", x_scale="stdz", ax=ax[0][1])
kline_prior_plot(b_prior="bespoke", x_scale="log", ax=ax[1][0])
kline_prior_plot(x_scale="natural", ax=ax[1][1])

#### Code 11.45

In [ ]:
# intercept only
with pm.Model() as m11_9:
    a = pm.Normal("a", 3.0, 0.5)
    T = pm.Poisson("total_tools", pm.math.exp(a), observed=dk.total_tools)
    trace_11_9 = pm.sample(tune=3000, random_seed=RANDOM_SEED)

# interaction model
with pm.Model() as m11_10:
    a = pm.Normal("a", 3.0, 0.5, shape=2)
    b = pm.Normal("b", 0.0, 0.2, shape=2)

    cid = pm.intX(pm.Data("cid", c_id))
    P_ = pm.Data("P", P)
    lam = pm.math.exp(a[cid] + b[cid] * P_)

    T = pm.Poisson("total_tools", lam, observed=dk.total_tools)
    trace_11_10 = pm.sample(tune=3000, random_seed=RANDOM_SEED)

#### Code 11.46

In [ ]:
az.compare({"m11_9": trace_11_9, "m11_10": trace_11_10})

In [ ]:
# store pareto-k values for plot:
k = az.loo(trace_11_10, pointwise=True).pareto_k.values

#### Code 11.47 and 11.48

In [ ]:
ns = 100
P_seq = np.linspace(-1.4, 3.0, ns)

with m11_10:
    # predictions for cid=0 (low contact)
    pm.set_data({"cid": np.array([0] * ns), "P": P_seq})
    lam0 = pm.sample_posterior_predictive(trace_11_10, var_names=["total_tools"])[
        "posterior_predictive"
    ]["total_tools"]

    # predictions for cid=1 (high contact)
    pm.set_data({"cid": np.array([1] * ns)})
    lam1 = pm.sample_posterior_predictive(trace_11_10, var_names=["total_tools"])[
        "posterior_predictive"
    ]["total_tools"]

lmu0, lmu1 = lam0.mean(["chain", "draw"]), lam1.mean(["chain", "draw"])

In [ ]:
_, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 6))

# scale point size to Pareto-k:
k /= k.max()
psize = 250 * k

# Plot on standardized log scale:

az.plot_hdi(P_seq, lam1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(P_seq, lmu1, color="b", alpha=0.7, label="high contact mean")

az.plot_hdi(P_seq, lam0, color="k", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(P_seq, lmu0, "--", color="k", alpha=0.7, label="low contact mean")

# display names and k:
mask = k > 0.3
labels = dk.culture.values[mask]
for i, text in enumerate(labels):
    ax0.text(
        P[mask][i] - 0.2,
        dk.total_tools.values[mask][i] + 4,
        f"{text} ({np.round(k[mask][i], 2)})",
        fontsize=8,
    )

# display observed data:
index = c_id == 1
ax0.scatter(
    P[~index],
    dk.total_tools[~index],
    s=psize[~index],
    facecolors="none",
    edgecolors="k",
    alpha=0.8,
    lw=1,
    label="low contact",
)
ax0.scatter(P[index], dk.total_tools[index], s=psize[index], alpha=0.8, label="high contact")
ax0.set_xlabel("log population (std)")
ax0.set_ylabel("total tools")
ax0.legend(fontsize=8, ncol=2)

# Plot on natural scale:
# unstandardize and exponentiate values of standardized log pop:
P_seq = np.linspace(-5.0, 3.0, ns)
P_seq = np.exp(P_seq * np.log(dk.population.values).std() + np.log(dk.population.values).mean())

az.plot_hdi(P_seq, lam1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(P_seq, lmu1, color="b", alpha=0.7)

az.plot_hdi(P_seq, lam0, color="k", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(P_seq, lmu0, "--", color="k", alpha=0.7)

# display observed data:
ax1.scatter(
    dk.population[~index],
    dk.total_tools[~index],
    s=psize[~index],
    facecolors="none",
    edgecolors="k",
    alpha=0.8,
    lw=1,
)
ax1.scatter(dk.population[index], dk.total_tools[index], s=psize[index], alpha=0.8)
plt.setp(ax1.get_xticklabels(), ha="right", rotation=45)
ax1.set_xlim((-10_000, 350_000))
ax1.set_xlabel("population")
ax1.set_ylabel("total tools");

#### Code 11.49
The book doesn't pre-process the population data, but if you give them raw to PyMC, the sampler will break: the scale of these data is too wide. However we can't just standardize the data, as we usually do. Why? Because some data points will then be negative, which doesn't play nice with the `b` exponent (try it if you don't trust me). But we'll do something similar: let's standardize the data, and then just add the absolute value of the minimum, and add yet again an epsilon -- this will ensure that our data stay positive and that the transformation will be easy to reverse when we want to plot on the natural scale:

In [ ]:
P = standardize(np.log(dk.population)).values
P = P + np.abs(P.min()) + 0.1  # must be > 0

And now we can run the model:

In [ ]:
with pm.Model() as m11_11:
    a = pm.Normal("a", 1.0, 1.0, shape=2)
    b = pm.Exponential("b", 1.0, shape=2)
    g = pm.Exponential("g", 1.0)

    cid = pm.intX(pm.Data("cid", c_id))
    P_ = pm.Data("P", P)
    lam = (at.exp(a[cid]) * P_ ** b[cid]) / g

    T = pm.Poisson("total_tools", lam, observed=dk.total_tools.values)
    trace_11_11 = pm.sample(2000, tune=2000, target_accept=0.9, random_seed=RANDOM_SEED)
az.plot_trace(trace_11_11, compact=True);

#### Bonus: posterior predictive plot for scientific model

In [ ]:
ns = 100
P_seq = np.linspace(-1.4, 3.0, ns) + 1.4  # our little trick

with m11_11:
    # predictions for cid=0 (low contact)
    pm.set_data({"cid": np.array([0] * ns), "P": P_seq})
    lam0 = pm.sample_posterior_predictive(trace_11_11, var_names=["total_tools"])[
        "posterior_predictive"
    ]["total_tools"]

    # predictions for cid=1 (high contact)
    pm.set_data({"cid": np.array([1] * ns)})
    lam1 = pm.sample_posterior_predictive(trace_11_11, var_names=["total_tools"])[
        "posterior_predictive"
    ]["total_tools"]

lmu0, lmu1 = lam0.mean(["chain", "draw"]), lam1.mean(["chain", "draw"])

In [ ]:
_, (ax0, ax1) = plt.subplots(1, 2, figsize=(14, 6))

# Plot on standardized log scale:
az.plot_hdi(P_seq, lam1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(P_seq, lmu1, color="b", alpha=0.7, label="high contact mean")

az.plot_hdi(P_seq, lam0, color="k", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(P_seq, lmu0, "--", color="k", alpha=0.7, label="low contact mean")

# display observed data:
index = c_id == 1
ax0.scatter(
    P[~index],
    dk.total_tools[~index],
    s=psize[~index],
    facecolors="none",
    edgecolors="k",
    alpha=0.8,
    lw=1,
    label="low contact",
)
ax0.scatter(P[index], dk.total_tools[index], s=psize[index], alpha=0.8, label="high contact")
ax0.set_xlabel("standardized population")
ax0.set_ylabel("total tools")
ax0.legend(fontsize=8, ncol=2)

# Plot on natural scale:
# unstandardize our log pop sequence:
P_seq = np.exp(
    (P_seq - 1.4) * np.log(dk.population.values).std() + np.log(dk.population.values).mean()
)

az.plot_hdi(P_seq, lam1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(P_seq, lmu1, color="b", alpha=0.7)

az.plot_hdi(P_seq, lam0, color="k", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(P_seq, lmu0, "--", color="k", alpha=0.7)

# display observed data:
ax1.scatter(
    dk.population[~index],
    dk.total_tools[~index],
    s=psize[~index],
    facecolors="none",
    edgecolors="k",
    alpha=0.8,
    lw=1,
)
ax1.scatter(dk.population[index], dk.total_tools[index], s=psize[index], alpha=0.8)
plt.setp(ax1.get_xticklabels(), ha="right", rotation=45)
ax1.set_xlim((-10_000, 350_000))
ax1.set_xlabel("population")
ax1.set_ylabel("total tools");

#### Code 11.50

In [ ]:
num_days = 30
y = np.random.poisson(1.5, num_days)
y

#### Code 11.51

In [ ]:
num_weeks = 4
y_new = np.random.poisson(0.5 * 7, num_weeks)
y_new

#### Code 11.52

In [ ]:
y_all = np.hstack([y, y_new])
exposure = np.hstack([np.repeat(1, 30), np.repeat(7, 4)]).astype("float")
monastery = np.hstack([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame({"y": y_all, "days": exposure, "monastery": monastery})
d

#### Code 11.53

In [ ]:
# compute the offset:
log_days = np.log(exposure)

# fit the model:
with pm.Model() as m11_12:
    a = pm.Normal("a", 0.0, 1.0)
    b = pm.Normal("b", 0.0, 1.0)

    lam = pm.math.exp(log_days + a + b * monastery)

    obs = pm.Poisson("y", lam, observed=y_all)

    trace_11_12 = pm.sample(1000, tune=2000, random_seed=RANDOM_SEED)

#### Code 11.54

In [ ]:
lambda_old = np.exp(trace_11_12["posterior"]["a"])
lambda_new = np.exp(trace_11_12["posterior"]["a"] + trace_11_12["posterior"]["b"])

az.summary({"lambda_old": lambda_old, "lambda_new": lambda_new}, kind="stats", round_to=2)

#### Code 11.55

In [ ]:
# simulate career choices among 500 individuals
N = 500  # number of individuals
income = np.array([1, 2, 5])  # expected income of each career
score = 0.5 * income  # score for each career, based on income
# converts scores to probabilities:
p = softmax(score)

# now simulate choice
# outcome career holds event type values, not counts
career = np.random.multinomial(1, p, size=N)
career = np.where(career == 1)[1]
career[:11], score, p

#### Code 11.56 and 11.57

The model described in the book does not sample well in PyMC. It does slightly better if we change the pivot category to be the first career instead of the third, but this is still suboptimal because we are discarding predictive information from the pivoted category (i.e., its unique career income). 

In fact, it is not necessary to pivot the coefficients of variables that are distinct for each category (what the author calls predictors matched to outcomes), as it is done for the coefficients of shared variables (what the author calles predictors matched to observations). The intercepts belong to the second category, and as such they still need to be pivoted. These two references explain this distinction clearly: 

* Hoffman, S. D., & Duncan, G. J. (1988). Multinomial and conditional logit discrete-choice models in demography. Demography, 25(3), 415-427 [pdf link](https://www.jstor.org/stable/pdf/2061541.pdf)
* Croissant, Y. (2020). Estimation of Random Utility Models in R: The mlogit Package. Journal of Statistical Software, 95(1), 1-41 [pdf link](https://www.jstatsoft.org/index.php/jss/article/view/v095i11/v95i11.pdf)

In [ ]:
with pm.Model() as m11_13:
    a = pm.Normal("a", 0.0, 1.0, shape=2)  # intercepts
    b = pm.HalfNormal("b", 0.5)  # association of income with choice

    s0 = a[0] + b * income[0]
    s1 = a[1] + b * income[1]
    s2 = 0.0 + b * income[2]  # pivoting the intercept for the third category
    s = pm.math.stack([s0, s1, s2])

    p_ = at.nnet.softmax(s)
    career_obs = pm.Categorical("career", p=p_, observed=career)

    trace_11_13 = pm.sample(tune=2000, target_accept=0.99, random_seed=RANDOM_SEED)
az.summary(trace_11_13, round_to=2)

#### Code 11.58

Because this model better matches the actual data generating process, it also does a better job at predicting the effect of doubling the income of the second career.

In [ ]:
# set up logit scores:
post_11_13 = az.extract_dataset(trace_11_13["posterior"])
s0 = post_11_13["a"][0, :] + post_11_13["b"] * income[0]
s1_orig = post_11_13["a"][1, :] + post_11_13["b"] * income[1]
s1_new = post_11_13["a"][1, :] + post_11_13["b"] * income[1] * 2
s2 = 0.0 + post_11_13["b"] * income[2]

pp_scores_orig = np.stack([np.array(s0), np.array(s1_orig), np.array(s2)]).T
pp_scores_new = np.stack([np.array(s0), np.array(s1_new), s2]).T

# compute probabilities for original and counterfactual:
p_orig = softmax(pp_scores_orig, axis=1)
print(p_orig.shape)
p_new = softmax(pp_scores_new, axis=1)
print(p_new.shape)

# summarize
p_diff = p_new[:, 1] - p_orig[:, 1]
az.summary({"p_diff": p_diff}, kind="stats", round_to=2)

In [ ]:
# Actual difference when doubling the income of option #2
income_orig = np.array((1, 2, 5))  # expected income of each career
score_orig = 0.5 * income  # scores for each career, based on income
p_orig = softmax(score_orig)

income_new = np.array((1, 2 * 2, 5))  # expected income of each career
score_new = 0.5 * income_new  # scores for each career, based on income
p_new = softmax(score_new)

print("True p_diff:", p_new[1] - p_orig[1])  # Change in probability of the second career

#### Code 11.59

In [ ]:
N = 500

# simulate family incomes for each individual
family_income = np.random.rand(N)

# assign a unique coefficient for each type of event
b = np.array([-2.0, 0.0, 2.0])

p = softmax(np.array([0.5, 1.0, 1.5])[:, None] + np.outer(b, family_income), axis=0).T

career = np.asarray([np.random.multinomial(1, pp) for pp in p])
career = np.where(career == 1)[1]
career[:11]

In [ ]:
with pm.Model() as m11_14:
    a = pm.Normal("a", 0.0, 1.5, shape=2)  # intercepts
    b = pm.Normal("b", 0.0, 1.0, shape=2)  # coefficients on family income

    s0 = a[0] + b[0] * family_income
    s1 = a[1] + b[1] * family_income
    s2 = np.zeros(N)  # pivot
    s = pm.math.stack([s0, s1, s2]).T

    p_ = at.nnet.softmax(s)
    career_obs = pm.Categorical("career", p=p_, observed=career)

    trace_11_14 = pm.sample(1000, tune=2000, target_accept=0.9, random_seed=RANDOM_SEED)
az.summary(trace_11_14, round_to=2)

#### Code 11.60

In [ ]:
d_ad = pd.read_csv("Data/UCBadmit.csv", sep=";")

#### Code 11.61

In [ ]:
# binomial model of overall admission probability
with pm.Model() as m_binom:
    a = pm.Normal("a", 0, 1.5)
    p = pm.math.invlogit(a)

    admit = pm.Binomial("admit", p=p, n=d_ad.applications.values, observed=d_ad.admit.values)

    trace_binom = pm.sample(1000, tune=2000)

# Poisson model of overall admission and rejection rates
with pm.Model() as m_pois:
    a = pm.Normal("a", 0, 1.5, shape=2)
    lam = pm.math.exp(a)

    admit = pm.Poisson("admit", lam[0], observed=d_ad.admit.values)
    rej = pm.Poisson("rej", lam[1], observed=d_ad.reject.values)

    trace_pois = pm.sample(1000, tune=2000)

#### Code 11.62

In [ ]:
m_binom = az.summary(trace_binom)
logistic(m_binom["mean"]).round(3)

#### Code 11.63

In [ ]:
m_pois = az.summary(trace_pois).round(2)
(np.exp(m_pois["mean"][0]) / (np.exp(m_pois["mean"][0]) + np.exp(m_pois["mean"][1]))).round(3)

11M7.  Use quap to construct a quadratic approximate posterior distribution for the chimpanzee model that includes a unique intercept for each actor, m11.4 (page 330). Compare the quadratic approximation to the posterior distribution produced instead from MCMC. Can you explain both the differences and the similarities between the approximate and the MCMC distributions? Relax the prior on the actor intercepts to Normal(0,10). Re-estimate the posterior using both ulam and quap. Do the differences increase or decrease? Why?

In [ ]:
d = pd.read_csv("Data/chimpanzees.csv", sep=";")
# we change "actor" to zero-index
d.actor = d.actor - 1

In [ ]:
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

In [ ]:
actor_idx, actors = pd.factorize(d.actor)
treat_idx, treatments = pd.factorize(d.treatment)

In [ ]:
with pm.Model() as m11_4:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    actor_id = pm.intX(pm.Data("actor_id", actor_idx))
    treat_id = pm.intX(pm.Data("treat_id", treat_idx))
    p = pm.Deterministic("p", pm.math.invlogit(a[actor_id] + b[treat_id]))

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

az.summary(trace_11_4, var_names=["a", "b"], round_to=2)

In [ ]:
#quadratic approximation

n_obs = d.shape[0]
n_actors = len(actor_idx)
n_treatments = len(treat_idx)

import pymc as pm
import pytensor.tensor as pt  # for tensor ops
from pytensor import function
from pytensor.gradient import hessian
from pytensor import function
# from pytensor.gradient import hessian
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal

# Define the model
with pm.Model() as model:
    a = pm.Normal("a", mu=0, sigma=1.5, shape=n_actors)
    b = pm.Normal("b", mu=0, sigma=0.5, shape=n_treatments)
    
    actor_id = pm.ConstantData("actor_id", actor_idx)
    treat_id = pm.ConstantData("treat_id", treat_idx)
    
    p = pm.Deterministic("p", pm.math.sigmoid(a[actor_id] + b[treat_id]))
    y = pm.Binomial("pulled_left", n=1, p=p, observed=d.pulled_left)

    # Find MAP estimate
    map_estimate = pm.find_MAP()

    # --- Quadratic approximation using Hessian ---

    # Get the value variables corresponding to the free RVs
    free_rvs = model.free_RVs
    value_vars = [model.rvs_to_values[rv] for rv in free_rvs]
    
    # 1. Create logp terms
    logps = [pm.logp(rv, value_var) for rv, value_var in zip(free_rvs, value_vars)]
    
    # 2. Total scalar logp
    logp_scalar = pt.sum(pt.stack(logps))
    
    # 3. Flatten all value variables for Hessian
    flat_vars = pt.concatenate([pt.reshape(v, (-1,)) for v in value_vars])
    
    # 4. Compute Hessian
    hess_expr = hessian(logp_scalar, flat_vars)
    hess_func = function(value_vars, hess_expr)
# 4. Evaluate Hessian at MAP
flat_map = pm.blockwise_flatten(map_estimate, free_rvs)
H = hess_func()

# 5. Compute covariance (Laplace approximation)
cov = np.linalg.inv(-H)

# 6. Sample from multivariate normal approximation
mean = np.concatenate([map_estimate["a"], map_estimate["b"]])
samples = multivariate_normal.rvs(mean=mean, cov=cov, size=1000)

# 7. Convert to DataFrame
param_names = [f"a[{i}]" for i in range(n_actors)] + [f"b[{j}]" for j in range(n_treatments)]
approx_df = pd.DataFrame(samples, columns=param_names)

gave up

11M8.  Revisit the data(Kline) islands example. This time drop Hawaii from the sample and refit the models. What changes do you observe?m

In [ ]:
dk = pd.read_csv("Data/Kline", sep=";")
dk

In [ ]:
dk.shape

In [ ]:
dk_wo_h=dk[dk.culture!='Hawaii']
dk_wo_h

In [ ]:
P_wo_h = standardize(np.log(dk_wo_h.population)).values
c_id_wo_h = (dk_wo_h.contact == "high").astype(int).values

In [ ]:
P = standardize(np.log(dk.population)).values
c_id = (dk.contact == "high").astype(int).values

In [ ]:
# intercept only
with pm.Model() as m11_9:
    a = pm.Normal("a", 3.0, 0.5)
    T = pm.Poisson("total_tools", pm.math.exp(a), observed=dk.total_tools)
    trace_11_9 = pm.sample(tune=3000, random_seed=RANDOM_SEED)

# interaction model
with pm.Model() as m11_10:
    a = pm.Normal("a", 3.0, 0.5, shape=2)
    b = pm.Normal("b", 0.0, 0.2, shape=2)

    cid = pm.intX(pm.Data("cid", c_id))
    P_ = pm.Data("P", P)
    lam = pm.math.exp(a[cid] + b[cid] * P_)

    T = pm.Poisson("total_tools", lam, observed=dk.total_tools)
    trace_11_10 = pm.sample(tune=3000, random_seed=RANDOM_SEED)

In [ ]:
az.summary(trace_11_9)

In [ ]:
az.summary(trace_11_10)

In [ ]:
# intercept only
with pm.Model() as m11_9_wo_h:
    a = pm.Normal("a", 3.0, 0.5)
    T = pm.Poisson("total_tools", pm.math.exp(a), observed=dk_wo_h.total_tools)
    trace_11_9_wo_h = pm.sample(tune=3000, random_seed=RANDOM_SEED)

In [ ]:
with pm.Model() as m11_10_wo_h:
    a = pm.Normal("a", 3.0, 0.5, shape=2)
    b = pm.Normal("b", 0.0, 0.2, shape=2)

    cid = pm.intX(pm.Data("cid", c_id_wo_h))
    P_ = pm.Data("P", P_wo_h)
    lam = pm.Deterministic("lam", pm.math.exp(a[cid] + b[cid] * P_))

    T = pm.Poisson("total_tools", lam, observed=dk_wo_h.total_tools)

    trace_11_10_wo_h = pm.sample(
        tune=3000,
        return_inferencedata=True,              # ✅ make sure to return InferenceData
        idata_kwargs={"log_likelihood": True},  # ✅ tell PyMC to include log_likelihood
        random_seed=RANDOM_SEED
    )

In [ ]:
az.summary(trace_11_9_wo_h)

In [ ]:
az.summary(trace_11_10_wo_h)

In [ ]:
ns = 100
P_seq = np.linspace(-1.4, 3.0, ns)

with m11_10_wo_h:
    pm.set_data({"cid": np.array([0] * ns), "P": P_seq})
    lam0_idata = pm.sample_posterior_predictive(trace_11_10_wo_h, var_names=["lam"])
    lam0 = lam0_idata.posterior_predictive["lam"]

    pm.set_data({"cid": np.array([1] * ns), "P": P_seq})
    lam1_idata = pm.sample_posterior_predictive(trace_11_10_wo_h, var_names=["lam"])
    lam1 = lam1_idata.posterior_predictive["lam"]

lmu0, lmu1 = lam0.mean(["chain", "draw"]), lam1.mean(["chain", "draw"])

# store pareto-k values for plot:
k = az.loo(trace_11_10_wo_h, pointwise=True).pareto_k.values

In [ ]:
_, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 6))

# scale point size to Pareto-k:
k /= k.max()
psize = 250 * k

# Plot on standardized log scale:

az.plot_hdi(P_seq, lam1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(P_seq, lmu1, color="b", alpha=0.7, label="high contact mean")

az.plot_hdi(P_seq, lam0, color="k", fill_kwargs={"alpha": 0.2}, ax=ax0)
ax0.plot(P_seq, lmu0, "--", color="k", alpha=0.7, label="low contact mean")

# display names and k:
mask = k > 0.3
labels = dk_wo_h.culture.values[mask]
for i, text in enumerate(labels):
    ax0.text(
        P_wo_h[mask][i] - 0.2,
        dk_wo_h.total_tools.values[mask][i] + 4,
        f"{text} ({np.round(k[mask][i], 2)})",
        fontsize=8,
    )

# display observed data:
index = c_id_wo_h == 1
ax0.scatter(
    P_wo_h[~index],
    dk_wo_h.total_tools[~index],
    s=psize[~index],
    facecolors="none",
    edgecolors="k",
    alpha=0.8,
    lw=1,
    label="low contact",
)
ax0.scatter(P_wo_h[index], dk_wo_h.total_tools[index], s=psize[index], alpha=0.8, label="high contact")
ax0.set_xlabel("log population (std)")
ax0.set_ylabel("total tools")
ax0.legend(fontsize=8, ncol=2)

# Plot on natural scale:
# unstandardize and exponentiate values of standardized log pop:
P_seq = np.linspace(-5.0, 3.0, ns)
P_seq = np.exp(P_seq * np.log(dk_wo_h.population.values).std() + np.log(dk_wo_h.population.values).mean())

az.plot_hdi(P_seq, lam1, color="b", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(P_seq, lmu1, color="b", alpha=0.7)

az.plot_hdi(P_seq, lam0, color="k", fill_kwargs={"alpha": 0.2}, ax=ax1)
ax1.plot(P_seq, lmu0, "--", color="k", alpha=0.7)

# display observed data:
ax1.scatter(
    dk_wo_h.population[~index],
    dk_wo_h.total_tools[~index],
    s=psize[~index],
    facecolors="none",
    edgecolors="k",
    alpha=0.8,
    lw=1,
)
ax1.scatter(dk_wo_h.population[index], dk_wo_h.total_tools[index], s=psize[index], alpha=0.8)
plt.setp(ax1.get_xticklabels(), ha="right", rotation=45)
ax1.set_xlim((-10_000, 80_000))
ax1.set_xlabel("population")
ax1.set_ylabel("total tools");

11H1.  Use WAIC or PSIS to compare the chimpanzee model that includes a unique intercept for each actor, m11.4 (page 330), to the simpler models fit in the same section. Interpret the results.

In [ ]:
with pm.Model() as m11_4:
    a = pm.Normal("a", 0.0, 1.5, shape=len(actors))
    b = pm.Normal("b", 0.0, 0.5, shape=len(treatments))

    actor_id = pm.intX(pm.Data("actor_id", actor_idx))
    treat_id = pm.intX(pm.Data("treat_id", treat_idx))
    p = pm.Deterministic("p", pm.math.invlogit(a[actor_id] + b[treat_id]))

    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_4 = pm.sample(random_seed=RANDOM_SEED)

az.summary(trace_11_4, var_names=["a", "b"], round_to=2)

In [ ]:
waic_m11_4 = pm.compute_log_likelihood(trace_11_4, model=m11_4)
waic_m11_4 = pm.waic(waic_m11_4)

In [ ]:
with pm.Model() as m11_3:
    a = pm.Normal("a", 0.0, 1.5)
    b = pm.Normal("b", 0.0, 0.5, shape=4)

    p = pm.math.invlogit(a + b[d.treatment.values])
    pulled_left = pm.Binomial("pulled_left", 1, p, observed=d.pulled_left)

    trace_11_3 = pm.sample(random_seed=RANDOM_SEED)

In [ ]:
waic_m11_3 = pm.compute_log_likelihood(trace_11_3, model=m11_3)
waic_m11_3 = pm.waic(waic_m11_3)
waic_m11_3

In [ ]:
waic_m11_4

11H2.  The data contained in library(MASS);data(eagles) are records of salmon pirating attempts by Bald Eagles in Washington State. See ?eagles for details. While one eagle feeds, sometimes another will swoop in and try to steal the salmon from it. Call the feeding eagle the “victim” and the thief the “pirate.” Use the available data to build a binomial GLM of successful pirating attempts.

(a)  Consider the following model:

 

where y is the number of successful attempts, n is the total number of attempts, P is a dummy variable indicating whether or not the pirate had large body size, V is a dummy variable indicating whether or not the victim had large body size, and finally A is a dummy variable indicating whether or not the pirate was an adult. Fit the model above to the eagles data, using both quap and ulam. Is the quadratic approximation okay?

(b)  Now interpret the estimates. If the quadratic approximation turned out okay, then it’s okay to use the quap estimates. Otherwise stick to ulam estimates. Then plot the posterior predictions. Compute and display both (1) the predicted probability. of success and its 89% interval for each row (i) in the data, as well as (2) the predicted success count. and its 89% interval. What different information does each type of posterior prediction provide?

(c)  Now try to improve the model. Consider an interaction between the pirate’s size and age (immature or adult). Compare this model to the previous one, using WAIC. Interpret.

In [ ]:
eagles_df = pd.DataFrame({
    "y":  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
    "n":  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    "P":  ["L"]*10 + ["S"]*10 + ["L"]*10 + ["S"]*10,
    "V":  ["L"]*5 + ["S"]*5 + ["L"]*5 + ["S"]*5 + ["L"]*5 + ["S"]*5 + ["L"]*5 + ["S"]*5,
    "A":  ["A"]*20 + ["I"]*20
})

In [ ]:
eagles_df

In [ ]:
eagles_df['success_rate']=eagles_df.y/eagles_df.n

v_idx, vs = pd.factorize(eagles_df.V)
p_idx, ps = pd.factorize(eagles_df.P)

In [ ]:
set(p_idx)

In [ ]:
with pm.Model() as m_eagles:
    a = pm.Normal("a", 0.0, 1.5)
    bp = pm.Normal("bp", 0.0, 0.5, shape=len(set(p_idx)))
    bv = pm.Normal("bv", 0.0, 0.5, shape=len(set(v_idx)))

    v_id = pm.intX(pm.Data("v_id", v_idx))
    p_id = pm.intX(pm.Data("p_id", p_idx))
    p = pm.Deterministic("p", pm.math.invlogit(a + bp[p_id] + bv[v_id]))
    n =pm.Data("n", eagles_df.n)

    fishing_success = pm.Binomial("fishing_success", n, p, observed=eagles_df.y)

    trace_eagles = pm.sample(random_seed=RANDOM_SEED)

(b)  Now interpret the estimates. If the quadratic approximation turned out okay, then it’s okay to use the quap estimates. Otherwise stick to ulam estimates. Then plot the posterior predictions. Compute and display both (1) the predicted probability. of success and its 89% interval for each row (i) in the data, as well as (2) the predicted success count. and its 89% interval. What different information does each type of posterior prediction provide?

In [ ]:
az.summary(trace_eagles)

In [ ]:
ax = az.plot_forest(trace_eagles,var_names=["a", "bp", "bv"], combined=True)

In [ ]:
az.plot_trace(trace_eagles)

In [ ]:
with m_eagles:
    pm.set_data({
        "v_id": v_idx,
        "p_id": p_idx,
        "n": eagles_df.n
    })
    ppd = pm.sample_posterior_predictive(trace_eagles, var_names=["fishing_success", "p"], random_seed=RANDOM_SEED)

In [ ]:
ppd

In [ ]:
# shape: (chains, draws, observations)
p_samples = ppd.posterior_predictive["p"]

fig, ax = plt.subplots(figsize=(8, 6))
az.plot_hdi(np.arange(len(eagles_df)), p_samples, hdi_prob=0.89, ax=ax)
ax.plot(np.arange(len(eagles_df)), p_samples.mean(dim=["chain", "draw"]), "o", label="Mean probability")
ax.set_xlabel("Observation index")
ax.set_ylabel("Predicted probability of success")
ax.set_title("Predicted success probabilities with 89% HDIs")
ax.legend()

In [ ]:
# Assuming ppd_p is from sample_posterior_predictive
p_samples = ppd_p.posterior_predictive["p"]  # shape: (chain, draw, obs)

# Convert to numpy array
p_array = p_samples.stack(sample=("chain", "draw")).values.T  # shape: (samples, obs)
# Assume p_array shape: (samples, n_observations)
n_obs = p_array.shape[1]
means = p_array.mean(axis=0)
hdi_lower, hdi_upper = az.hdi(p_array, hdi_prob=0.89).T  # shape: (n_obs,)

# Plot
fig, ax = plt.subplots(figsize=(6, n_obs * 0.4))
y_pos = np.arange(n_obs)

ax.errorbar(
    x=means, 
    y=y_pos, 
    xerr=[means - hdi_lower, hdi_upper - means],
    fmt="o", 
    color="blue", 
    ecolor="lightblue", 
    capsize=4
)

ax.set_yticks(y_pos)
ax.set_yticklabels([f"obs {i}" for i in y_pos])
ax.set_xlabel("Predicted success probability")
ax.set_title("Posterior predictive success probabilities (with 89% HDI)")
plt.tight_layout()
plt.show()

In [ ]:
# Assuming ppd_p is from sample_posterior_predictive
p_samples = ppd.posterior_predictive["fishing_success"]  # shape: (chain, draw, obs)

# Convert to numpy array
p_array = p_samples.stack(sample=("chain", "draw")).values.T  # shape: (samples, obs)
# Assume p_array shape: (samples, n_observations)
n_obs = p_array.shape[1]
means = p_array.mean(axis=0)
hdi_lower, hdi_upper = az.hdi(p_array, hdi_prob=0.89).T  # shape: (n_obs,)

# Plot
fig, ax = plt.subplots(figsize=(6, n_obs * 0.4))
y_pos = np.arange(n_obs)

ax.errorbar(
    x=means, 
    y=y_pos, 
    xerr=[means - hdi_lower, hdi_upper - means],
    fmt="o", 
    color="blue", 
    ecolor="lightblue", 
    capsize=4
)

ax.set_yticks(y_pos)
ax.set_yticklabels([f"obs {i}" for i in y_pos])
ax.set_xlabel("Predicted success count")
ax.set_title("Posterior predictive success count (with 89% HDI)")
plt.tight_layout()
plt.show()

In [ ]:
# Assuming ppd_p is from sample_posterior_predictive
p_samples = ppd.posterior_predictive["p"]  # shape: (chain, draw, obs)

# Convert to numpy array
p_array = p_samples.stack(sample=("chain", "draw")).values.T  # shape: (samples, obs)
# Group predictions by pirate size
groups = np.unique(p_idx)
group_names = ["small", "large"]
group_hdis = []
group_means = []

for g in groups:
    group_samples = p_array[:, p_idx == g].flatten()  # flatten across all matching rows
    hdi = az.hdi(group_samples, hdi_prob=0.89)
    mean = np.mean(group_samples)
    group_hdis.append(hdi)
    group_means.append(mean)

# Plot
fig, ax = plt.subplots()
y = np.arange(len(groups))
hdil, hdiu = np.array(group_hdis).T
ax.errorbar(
    x=group_means,
    y=y,
    xerr=[group_means - hdil, hdiu - group_means],
    fmt="o",
    capsize=4,
    color="darkblue",
    ecolor="skyblue"
)
ax.set_yticks(y)
ax.set_yticklabels([f"Pirate size: {n}" for n in group_names])
ax.set_xlabel("Predicted success probability")
ax.set_title("Posterior predictive mean + 89% HDI by Pirate Size")
plt.tight_layout()
plt.show()

In [ ]:
# Assuming ppd_p is from sample_posterior_predictive
p_samples = ppd.posterior_predictive["p"]  # shape: (chain, draw, obs)

# Convert to numpy array
p_array = p_samples.stack(sample=("chain", "draw")).values.T  # shape: (samples, obs)
# Group predictions by pirate size
groups = np.unique(v_idx)
group_names = ["small", "large"]
group_hdis = []
group_means = []

for g in groups:
    group_samples = p_array[:, v_idx == g].flatten()  # flatten across all matching rows
    hdi = az.hdi(group_samples, hdi_prob=0.89)
    mean = np.mean(group_samples)
    group_hdis.append(hdi)
    group_means.append(mean)

# Plot
fig, ax = plt.subplots()
y = np.arange(len(groups))
hdil, hdiu = np.array(group_hdis).T
ax.errorbar(
    x=group_means,
    y=y,
    xerr=[group_means - hdil, hdiu - group_means],
    fmt="o",
    capsize=4,
    color="darkblue",
    ecolor="skyblue"
)
ax.set_yticks(y)
ax.set_yticklabels([f"Pirate size: {n}" for n in group_names])
ax.set_xlabel("Predicted success probability")
ax.set_title("Posterior predictive mean + 89% HDI by Victim Size")
plt.tight_layout()
plt.show()

(c)  Now try to improve the model. Consider an interaction between the pirate’s size and age (immature or adult). Compare this model to the previous one, using WAIC. Interpret.

In [ ]:
a_idx, a_s = pd.factorize(eagles_df.A)
eagles_df["p_a_nteraction"] = p_idx + 2 * a_idx
eagles_df['a_idx']=a_idx
eagles_df['p_idx']=p_idx
eagles_df.head()

In [ ]:
eagles_df.p_a_nteraction.value_counts()

In [ ]:
eagles_df

In [ ]:
with pm.Model() as m_eagles_inter:
    a = pm.Normal("a", 0.0, 1.5)
    bp = pm.Normal("bp", 0.0, 0.5, shape=len(set(p_idx)))
    ba = pm.Normal("ba", 0.0, 0.5, shape=len(set(a_idx)))
    b_ap =  pm.Normal("b_bp", 0.0, 0.5, shape=len(set(eagles_df["p_a_nteraction"])))

    a_id = pm.intX(pm.Data("a_id", a_idx))
    p_id = pm.intX(pm.Data("p_id", p_idx))
    pa_id = pm.intX(pm.Data("pa_id", eagles_df.p_a_nteraction))
    p = pm.Deterministic("p", pm.math.invlogit(a + bp[p_id] + ba[a_id] + b_ap[pa_id]))
    n =pm.Data("n", eagles_df.n)

    fishing_success = pm.Binomial("fishing_success", n, p, observed=eagles_df.y)

    trace_eagles_inter = pm.sample(random_seed=RANDOM_SEED)

In [ ]:
az.plot_trace(trace_eagles_inter)

In [ ]:
az.summary(trace_eagles_inter)

In [ ]:
with pm.Model() as m_eagles_inter:
    a = pm.Normal("a", 0.0, 1.5)
    bp = pm.Normal("bp", 0.0, 0.5, shape=len(set(p_idx)))
    ba = pm.Normal("ba", 0.0, 0.5, shape=len(set(a_idx)))
    b_ap =  pm.Normal("b_bp", 0.0, 0.5, shape=len(set(eagles_df["p_a_nteraction"])))

    a_id = pm.intX(pm.Data("a_id", a_idx))
    p_id = pm.intX(pm.Data("p_id", p_idx))
    pa_id = pm.intX(pm.Data("pa_id", eagles_df.p_a_nteraction))
    p = pm.Deterministic("p", pm.math.invlogit(a + bp[p_id] + ba[a_id] + b_ap[pa_id]))
    n =pm.Data("n", eagles_df.n)

    fishing_success = pm.Binomial("fishing_success", n, p, observed=eagles_df.y)
    prior_m_eagles_inter = pm.sample_prior_predictive(random_seed=RANDOM_SEED)

In [ ]:
prior_m_eagles_inter.prior['b_bp']
az.plot_density(prior_m_eagles_inter.prior, var_names=['b_bp'])

In [ ]:
pa_id

In [ ]:
with m_eagles_inter:
    pm.set_data({
        "pa_id": eagles_df.p_a_nteraction,
        "p_id": p_idx,
        "n": eagles_df.n
    })
    ppd_inter = pm.sample_posterior_predictive(trace_eagles, var_names=["fishing_success", "p"], random_seed=RANDOM_SEED)

In [ ]:
# Assuming ppd_p is from sample_posterior_predictive
p_samples = ppd_inter.posterior_predictive["fishing_success"]  # shape: (chain, draw, obs)

# Convert to numpy array
p_array = p_samples.stack(sample=("chain", "draw")).values.T  # shape: (samples, obs)
# Assume p_array shape: (samples, n_observations)
n_obs = p_array.shape[1]
means = p_array.mean(axis=0)
hdi_lower, hdi_upper = az.hdi(p_array, hdi_prob=0.89).T  # shape: (n_obs,)

# Plot
fig, ax = plt.subplots(figsize=(6, n_obs * 0.4))
y_pos = np.arange(n_obs)

ax.errorbar(
    x=means, 
    y=y_pos, 
    xerr=[means - hdi_lower, hdi_upper - means],
    fmt="o", 
    color="blue", 
    ecolor="lightblue", 
    capsize=4
)

ax.set_yticks(y_pos)
ax.set_yticklabels([f"obs {i}" for i in y_pos])
ax.set_xlabel("Predicted success count")
ax.set_title("Posterior predictive success count (with 89% HDI)")
plt.tight_layout()
plt.show()

In [ ]:
eagles_df

In [ ]:
# Assuming ppd_p is from sample_posterior_predictive
p_samples = ppd_inter.posterior_predictive["p"]  # shape: (chain, draw, obs)

# Convert to numpy array
p_array = p_samples.stack(sample=("chain", "draw")).values.T  # shape: (samples, obs)
# Group predictions by pirate size
groups = np.unique(eagles_df.p_a_nteraction)
group_names = ["Age=A, Body=Small", "Age=A, Body=Large","Age=I, Body=Small" ,"Age=I, Body=Large"]
group_hdis = []
group_means = []

for g in groups:
    group_samples = p_array[:, eagles_df.p_a_nteraction == g].flatten()  # flatten across all matching rows
    hdi = az.hdi(group_samples, hdi_prob=0.89)
    mean = np.mean(group_samples)
    group_hdis.append(hdi)
    group_means.append(mean)

# Plot
fig, ax = plt.subplots()
y = np.arange(len(groups))
hdil, hdiu = np.array(group_hdis).T
ax.errorbar(
    x=group_means,
    y=y,
    xerr=[group_means - hdil, hdiu - group_means],
    fmt="o",
    capsize=4,
    color="darkblue",
    ecolor="skyblue"
)
ax.set_yticks(y)
ax.set_yticklabels([f"Pirate size: {n}" for n in group_names])
ax.set_xlabel("Predicted success probability")
ax.set_title("Posterior predictive mean + 89% HDI by Pirate Size")
plt.tight_layout()
plt.show()

In [ ]:
waic_eagles_inter = pm.compute_log_likelihood(trace_eagles_inter, model=m_eagles_inter)
waic_eagles_inter = pm.waic(waic_eagles_inter)

In [ ]:
waic_eagles = pm.compute_log_likelihood(trace_eagles, model=m_eagles)
waic_eagles = pm.waic(waic_eagles)

In [ ]:
waic_eagles_inter

In [ ]:
waic_eagles

looks like that interaction model has improved the prediction accuracy of the model

11H3.  The data contained in data(salamanders) are counts of salamanders (Plethodon elongatus) from 47 different 49-m2 plots in northern California.181 The column SALAMAN is the count in each plot, and the columns PCTCOVER and FORESTAGE are percent of ground cover and age of trees in the plot, respectively. You will model SALAMAN as a Poisson variable.

(a)  Model the relationship between density and percent cover, using a log-link (same as the example in the book and lecture). Use weakly informative priors of your choosing. Check the quadratic approximation again, by comparing quap to ulam. Then plot the expected counts and their 89% interval against percent cover. In which ways does the model do a good job? A bad job?

(b)  Can you improve the model by using the other predictor, FORESTAGE? Try any models you think useful. Can you explain why FORESTAGE helps or does not help with prediction?

In [ ]:
df_salam=pd.read_csv('End_of_chapter_problems/data/salamanders.csv', sep=';')
df_salam.shape

In [ ]:
df_salam.head()

In [ ]:
df_salam.SALAMAN.hist()

In [ ]:
df_salam.SALAMAN.mean(), np.log(df_salam.SALAMAN.mean())

In [ ]:
df_salam.SALAMAN.value_counts()

In [ ]:
df_salam.PCTCOVER.hist()

In [ ]:
df_salam.FORESTAGE.hist()

In [ ]:
df_salam['PCTCOVER_std']=standardize(df_salam.PCTCOVER)
df_salam['FORESTAGE_std']=standardize(df_salam.FORESTAGE)

In [ ]:
with pm.Model() as m_salam:
    a = pm.Normal("a", 0.0, 0.6)
    b_cover = pm.Normal("b_cover", 0.0, 0.2)
    # b_forestage = pm.Normal("b_forestage", 0.0, 0.2)
    
    cover_data = pm.Data("cover_data", df_salam.PCTCOVER_std)
    # forestage_data = pm.Data("forestage_data", df_salam.FORESTAGE_std)
    # lam = pm.math.exp(a + b_cover * cover_data + b_forestage*forestage_data)
    lam = pm.math.exp(a + b_cover * cover_data)

    salam_count = pm.Poisson("salam_count", lam, observed=df_salam.SALAMAN)
    trace_m_salam = pm.sample(random_seed=RANDOM_SEED)
    prior_m_salam = pm.sample_prior_predictive(random_seed=RANDOM_SEED)

#### priors

In [ ]:
az_prior = az.from_dict(prior_predictive={"salam_count": prior_m_salam.prior_predictive["salam_count"]})

az.plot_dist(
    az_prior.prior_predictive["salam_count"].stack(samples=("chain", "draw")),
    kind="hist",
    hist_kwargs={'bins':10},
    color="lightcoral",
    backend_kwargs={"figsize": (7, 4)}
)
plt.xlabel("Simulated salamander counts")
plt.title("Prior Predictive Distribution of Salamander Counts")
plt.show()

In [ ]:
az.plot_density(prior_m_salam.prior['a'])

In [ ]:
az.plot_density(prior_m_salam.prior['b_cover'])

In [ ]:
az_prior = az.from_dict(prior={k: v for k, v in prior_m_salam.prior.items()})

az.plot_posterior(
    az_prior,
    var_names=["a", "b_cover"],
    kind="hist",
    hdi_prob=0.89,
    group="prior",
    figsize=(8, 3)
)

In [ ]:
az.plot_trace(trace_m_salam)

In [ ]:
az.summary(trace_m_salam2)

In [ ]:
P_seq = np.linspace(df_salam["PCTCOVER_std"].min(), df_salam["PCTCOVER_std"].max(), 47)

with m_salam:
    pm.set_data({"cover_data": P_seq})
    lam_pred = pm.sample_posterior_predictive(trace_m_salam, var_names=["salam_count"])['posterior_predictive']['salam_count']

# Compute mean and 89% HDI
mean_lam = lam_pred.mean(dim=('chain', 'draw'))
hdi_lam = az.hdi(lam_pred, hdi_prob=0.89)

# Get HDI as NumPy array from the xarray DataArray
hdi_array = hdi_lam["salam_count"].values

plt.figure(figsize=(8, 4))
plt.plot(P_seq, mean_lam, label="Expected count")
plt.fill_between(P_seq, hdi_array[:, 0], hdi_array[:, 1], alpha=0.3, label="89% HDI")
plt.xlabel("Standardized % cover")
plt.ylabel("Expected salamander count")
plt.legend()


(b)  Can you improve the model by using the other predictor, FORESTAGE? Try any models you think useful. Can you explain why FORESTAGE helps or does not help with prediction?

In [ ]:
with pm.Model() as m_salam2:
    a = pm.Normal("a", 0.0, 0.6)
    b_cover = pm.Normal("b_cover", 0.0, 0.2)
    b_forestage = pm.Normal("b_forestage", 0.0, 0.2)
    
    cover_data = pm.Data("cover_data", df_salam.PCTCOVER_std)
    forestage_data = pm.Data("forestage_data", df_salam.FORESTAGE_std)
    lam = pm.math.exp(a + b_cover * cover_data + b_forestage*forestage_data)
    # lam = pm.math.exp(a + b_cover * cover_data)

    salam_count = pm.Poisson("salam_count", lam, observed=df_salam.SALAMAN)
    trace_m_salam2 = pm.sample(random_seed=RANDOM_SEED)
    prior_m_salam2 = pm.sample_prior_predictive(random_seed=RANDOM_SEED)

In [ ]:
az_prior = az.from_dict(prior_predictive={"salam_count": prior_m_salam2.prior_predictive["salam_count"]})

az.plot_dist(
    az_prior.prior_predictive["salam_count"].stack(samples=("chain", "draw")),
    kind="hist",
    hist_kwargs={'bins':10},
    color="lightcoral",
    backend_kwargs={"figsize": (7, 4)}
)
plt.xlabel("Simulated salamander counts")
plt.title("Prior Predictive Distribution of Salamander Counts")
plt.show()

In [ ]:
az_prior = az.from_dict(prior={k: v for k, v in prior_m_salam2.prior.items()})

az.plot_posterior(
    az_prior,
    var_names=["a", "b_cover", "b_forestage"],
    kind="hist",
    hdi_prob=0.89,
    group="prior",
    figsize=(8, 3)
)

In [ ]:
az.plot_trace(trace_m_salam2)

In [ ]:
az.summary(trace_m_salam2)

In [ ]:
P_seq = np.linspace(df_salam["PCTCOVER_std"].min(), df_salam["PCTCOVER_std"].max(), 47)

with m_salam2:
    pm.set_data({"cover_data": P_seq})
    lam_pred = pm.sample_posterior_predictive(trace_m_salam2, var_names=["salam_count"])['posterior_predictive']['salam_count']

# Compute mean and 89% HDI
mean_lam = lam_pred.mean(dim=('chain', 'draw'))
hdi_lam = az.hdi(lam_pred, hdi_prob=0.89)

# Get HDI as NumPy array from the xarray DataArray
hdi_array = hdi_lam["salam_count"].values

plt.figure(figsize=(8, 4))
plt.plot(P_seq, mean_lam, label="Expected count")
plt.fill_between(P_seq, hdi_array[:, 0], hdi_array[:, 1], alpha=0.3, label="89% HDI")
plt.xlabel("Standardized cover")
plt.ylabel("Expected salamander count")
plt.legend()

In [ ]:
P_seq = np.linspace(df_salam["FORESTAGE_std"].min(), df_salam["FORESTAGE_std"].max(), 47)

with m_salam2:
    pm.set_data({"forestage_data": P_seq})
    lam_pred = pm.sample_posterior_predictive(trace_m_salam2, var_names=["salam_count"])['posterior_predictive']['salam_count']

# Compute mean and 89% HDI
mean_lam = lam_pred.mean(dim=('chain', 'draw'))
hdi_lam = az.hdi(lam_pred, hdi_prob=0.89)

# Get HDI as NumPy array from the xarray DataArray
hdi_array = hdi_lam["salam_count"].values

plt.figure(figsize=(8, 4))
plt.plot(P_seq, mean_lam, label="Expected count")
plt.fill_between(P_seq, hdi_array[:, 0], hdi_array[:, 1], alpha=0.3, label="89% HDI")
plt.xlabel("Standardized forest age")
plt.ylabel("Expected salamander count")
plt.legend()

In [ ]:
waic_salam = pm.compute_log_likelihood(trace_m_salam, model=m_salam)
waic_salam = pm.waic(waic_salam)

In [ ]:
waic_salam2 = pm.compute_log_likelihood(trace_m_salam2, model=m_salam2)
waic_salam2 = pm.waic(waic_salam2)

In [ ]:
waic_salam

In [ ]:
waic_salam2

If too many k values > 0.7, prefer PSIS-LOO over WAIC using az.loo().

In [ ]:
# Compute LOO (leave-one-out cross-validation)
loo_result = az.loo(trace_m_salam, pointwise=True)

# Now plot the Pareto-k diagnostics
az.plot_khat(loo_result)

In [ ]:
# Compute LOO (leave-one-out cross-validation)
loo_result2 = az.loo(trace_m_salam2, pointwise=True)

# Now plot the Pareto-k diagnostics
az.plot_khat(loo_result2)

In [ ]:
loo_result

In [ ]:
loo_result2

In [ ]:
with pm.Model() as m_salam3:
    a = pm.Normal("a", 0.0, 0.6)
    b_cover = pm.Normal("b_cover", 0.0, 0.2)
    b_forestage = pm.Normal("b_forestage", 0.0, 0.2)
    b_interact = pm.Normal("b_interact", 0.0, 0.1)
    
    cover_data = pm.Data("cover_data", df_salam.PCTCOVER_std)
    forestage_data = pm.Data("forestage_data", df_salam.FORESTAGE_std)
    lam_ = pm.math.exp(a + b_cover * cover_data + b_forestage*forestage_data+ b_interact*cover_data*forestage_data)
    lam = pm.Deterministic("lam", lam)

    salam_count = pm.Poisson("salam_count", lam_, observed=df_salam.SALAMAN)
    trace_m_salam3 = pm.sample(random_seed=RANDOM_SEED)
    prior_m_salam3 = pm.sample_prior_predictive(random_seed=RANDOM_SEED)

In [ ]:
az.summary(trace_m_salam3)

In [ ]:
az.plot_trace(trace_m_salam3)

In [ ]:
P_seq = np.linspace(df_salam["PCTCOVER_std"].min(), df_salam["PCTCOVER_std"].max(), 47)

with m_salam3:
    pm.set_data({"cover_data": P_seq})
    lam_pred = pm.sample_posterior_predictive(trace_m_salam3, var_names=["salam_count"])['posterior_predictive']['salam_count']

# Compute mean and 89% HDI
mean_lam = lam_pred.mean(dim=('chain', 'draw'))
hdi_lam = az.hdi(lam_pred, hdi_prob=0.89)

# Get HDI as NumPy array from the xarray DataArray
hdi_array = hdi_lam["salam_count"].values

plt.figure(figsize=(8, 4))
plt.plot(P_seq, mean_lam, label="Expected count")
plt.fill_between(P_seq, hdi_array[:, 0], hdi_array[:, 1], alpha=0.3, label="89% HDI")
plt.xlabel("Standardized cover")
plt.ylabel("Expected salamander count")
plt.legend()

In [ ]:
# Compute LOO (leave-one-out cross-validation)
waic_salam3 = pm.compute_log_likelihood(trace_m_salam3, model=m_salam3)
loo_result3 = az.loo(trace_m_salam3, pointwise=True)

# Now plot the Pareto-k diagnostics
az.plot_khat(loo_result3)

In [ ]:
loo_result3

In [ ]:
#nteraction grid, need this model wo observed data because it has then fixed number of datapoints
with pm.Model() as m_salam3_pred:
    a = pm.Normal("a", 0.0, 0.6)
    b_cover = pm.Normal("b_cover", 0.0, 0.2)
    b_forestage = pm.Normal("b_forestage", 0.0, 0.2)
    b_interact = pm.Normal("b_interact", 0.0, 0.1)
    
    cover_data = pm.Data("cover_data", cover_flat)
    forestage_data = pm.Data("forestage_data", forestage_flat)
    
    lam = pm.math.exp(a + b_cover * cover_data + b_forestage * forestage_data + b_interact * cover_data * forestage_data)
    
    salam_count = pm.Poisson("salam_count", lam)  # <- no observed here!

    # Use previously sampled posterior
    lam_pred = pm.sample_posterior_predictive(trace_m_salam3, var_names=["salam_count"], random_seed=RANDOM_SEED)

In [ ]:
# Create grid of standardized cover and forestage
cover_grid = np.linspace(df_salam["PCTCOVER_std"].min(), df_salam["PCTCOVER_std"].max(), 30)
forestage_grid = np.linspace(df_salam["FORESTAGE_std"].min(), df_salam["FORESTAGE_std"].max(), 30)
cover_mesh, forestage_mesh = np.meshgrid(cover_grid, forestage_grid)

# Flatten for prediction
cover_flat = cover_mesh.ravel()
forestage_flat = forestage_mesh.ravel()

# Extract the posterior predictive mean
mean_pred = lam_pred["posterior_predictive"]["salam_count"].mean(axis=(0, 1))  # shape (900,)
mean_grid = mean_pred.values.reshape(cover_mesh.shape)


In [ ]:
plt.figure(figsize=(8, 6))
contour = plt.contourf(cover_mesh, forestage_mesh, mean_grid, levels=20, cmap="viridis")
cbar = plt.colorbar(contour)
cbar.set_label("Expected salamander count")
plt.xlabel("Standardized % Cover")
plt.ylabel("Standardized Forest Age")
plt.title("Interaction Effect: Cover × Forest Age")
plt.show()

interaction model increases prediction accuracy but interaction term is negative meaning that if forest is older or coverage is wider it reduces a bit other term effect

11H4.  The data in data(NWOGrants) are outcomes for scientific funding applications for the Netherlands Organization for Scientific Research (NWO) from 2010–2012 (see van der Lee and Ellemers (2015) for data and context). These data have a very similar structure to the UCBAdmit data discussed in the chapter. I want you to consider a similar question: What are the total and indirect causal effects of gender on grant awards? Consider a mediation path (a pipe) through discipline. Draw the corresponding DAG and then use one or more binomial GLMs to answer the question. What is your causal interpretation? If NWO’s goal is to equalize rates of funding between men and women, what type of intervention would be most effective?

In [ ]:
df_grants=pd.read_csv('End_of_chapter_problems/data/NWOGrants.csv', sep=';')
df_grants.shape

In [ ]:
df_grants.head()

In [ ]:
df_grants['awards_p']=df_grants.awards/df_grants.applications
disc_ids = pd.Categorical(df_grants["discipline"]).codes
gender_ids = pd.Categorical(df_grants["gender"]).codes
df_grants['gender_id']=gender_ids
df_grants['discipline_id']=disc_ids
# gender_id

In [ ]:
df_grants.groupby('gender')['awards_p'].mean()

In [ ]:
summary = df_grants.groupby("gender").agg(
    total_applications=("applications", "sum"),
    total_awards=("awards", "sum")
)
summary["success_rate"] = summary["total_awards"] / summary["total_applications"]

summary

In [ ]:
df_grants.groupby('gender')

In [ ]:
df_grants.head()

In [ ]:
len(np.unique(gender_ids))

In [ ]:
(df_grants.awards_p).mean()

In [ ]:
#a prior is in logit scale
logit(0.19)

In [ ]:
df_grants.awards_p

In [ ]:
with pm.Model() as m_grants1:
    a = pm.Normal("a", -1.5, 0.5, shape=len(np.unique(gender_ids)))
    gender_id=pm.Data("gender_id", df_grants['gender_id'])
    p = pm.Deterministic("p", pm.math.invlogit(a[gender_id]))
    
    admit = pm.Binomial("award_p", p=p, n=df_grants.applications.values, observed=df_grants.awards)

    trace_m_grants1 = pm.sample(2000, random_seed=RANDOM_SEED)
    prior_m_grants1 = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
    
az.summary(trace_m_grants1, round_to=2)

In [ ]:
az_prior = az.from_dict(prior={k: v for k, v in prior_m_grants1.prior.items()})

az.plot_posterior(
    az_prior,
    var_names=["a", "p"],
    kind="hist",
    hdi_prob=0.89,
    group="prior",
    figsize=(12, 7)
)

In [ ]:
male_prob = expit(trace_m_grants1.posterior["a"].sel(a_dim_0=1)).mean().values
female_prob = expit(trace_m_grants1.posterior["a"].sel(a_dim_0=0)).mean().values

print(f"female ≈ {female_prob:.3f}, male ≈ {male_prob:.3f}")

In [ ]:
with pm.Model() as m_grants2:
    a = pm.Normal("a", -1.5, 0.5, shape=len(np.unique(gender_ids)))
    b_disc = pm.Normal("b_disc", 0, 0.5, shape=len(np.unique(disc_ids)))

    gender_id=pm.Data("gender_id", df_grants['gender_id'])
    disc_id=pm.Data("disc_id", df_grants['discipline_id'])
    p = pm.math.invlogit(a[gender_id] + b_disc[disc_id] )

    admit = pm.Binomial("award_p", p=p, n=df_grants.applications.values, observed=df_grants.awards)

    trace_m_grants2 = pm.sample(2000, random_seed=RANDOM_SEED)
    
az.summary(trace_m_grants2, round_to=2)

In [ ]:
#b_disc[0]
expit(0.38)

In [ ]:
#b_disc[7]
expit(-0.34)

In [ ]:
az.plot_forest(trace_m_grants2, combined=True)

In [ ]:
az.plot_forest(trace_m_grants1, combined=True, var_names=["a"])

In [ ]:
az.plot_forest(trace_m_grants2, combined=True, var_names=["a"])

In [ ]:
male_prob = expit(trace_m_grants2.posterior["a"].sel(a_dim_0=1)).mean().values
female_prob = expit(trace_m_grants2.posterior["a"].sel(a_dim_0=0)).mean().values

print(f"female ≈ {female_prob:.3f}, male ≈ {male_prob:.3f}")

In [ ]:
df_grants[df_grants.discipline_id.isin([0,7])]

So in male total gender effect is 0.177, but if controlling for discipline it is 0.192, which means that discipline might amplify some of the effects.  It is similar with female from 0.150 to 0.171. Alternative explanation is that there could be some third variable that affects result and gender (directly or via mediation). Male awards probability tends to be higher than female one. From disciplines 7 and 0 seem to have coefficients above/below 0. Discipline with  0 index is "Chemical sciences" and with index 7 is "Social sciences". In first award p tends to be higher in latter lower (note these coeffiecients are pooled, information is shared between disciplines). Most effective interventions would be increase number of women in the disciplines that have more awards given out.

In [ ]:
with pm.Model() as m_grants3:
    a = pm.Normal("a", -1.5, 0.5, shape=len(np.unique(gender_ids)))
    b_disc = pm.Normal("b_disc", 0, 0.5, shape=len(np.unique(disc_ids)))
    b_inter = pm.Normal("b_inter", 0, 0.3, shape=(len(np.unique(gender_ids)), len(np.unique(disc_ids))))  # interaction

    gender_id=pm.Data("gender_id", df_grants['gender_id'])
    disc_id=pm.Data("disc_id", df_grants['discipline_id'])

    p = pm.math.invlogit(a[gender_id] + b_disc[disc_id]+ b_inter[gender_id, disc_id] )

    admit = pm.Binomial("award_p", p=p, n=df_grants.applications.values, observed=df_grants.awards)

    trace_m_grants3 = pm.sample(2000, random_seed=RANDOM_SEED)
    
az.summary(trace_m_grants3, round_to=2)

In [ ]:
#interaction term coefficients were not significant

11H5.  Suppose that the NWO Grants sample has an unobserved confound that influences both choice of discipline and the probability of an award. One example of such a confound could be the career stage of each applicant. Suppose that in some disciplines, junior scholars apply for most of the grants. In other disciplines, scholars from all career stages compete. As a result, career stage influences discipline as well as the probability of being awarded a grant. Add these influences to your DAG from the previous problem. What happens now when you condition on discipline? Does it provide an un-confounded estimate of the direct path from gender to an award? Why or why not? Justify your answer with the backdoor criterion. If you have trouble thinking this though, try simulating fake data, assuming your DAG is true. Then analyze it using the model from the previous problem. What do you conclude? Is it possible for gender to have a real direct causal influence but for a regression conditioning on both gender and discipline to suggest zero influence?

Conditioning on discipline without also adjusting for career stage leads to a confounded estimate of the direct effect of gender on awards. According to the backdoor criterion, we must block all backdoor paths from the treatment (gender) to the outcome (award), and here, the only way to do that is to also condition on career stage — which we cannot do if it is unobserved

11H6.  The data in data(Primates301) are 301 primate species and associated measures. In this problem, you will consider how brain size is associated with social learning. There are three parts.

(a)  Model the number of observations of social_learning for each species as a function of the log brain size. Use a Poisson distribution for the social_learning outcome variable. Interpret the resulting posterior. (b) Some species are studied much more than others. So the number of reported instances of social_learning could be a product of research effort. Use the research_effort variable, specifically its logarithm, as an additional predictor variable. Interpret the coefficient for log research_effort. How does this model differ from the previous one? (c) Draw a DAG to represent how you think the variables social_learning, brain, and research_effort interact. Justify the DAG with the measured associations in the two models above (and any other models you used).

In [ ]:
df_primates=pd.read_csv('Data/Primates301.csv', sep=';')
df_primates.shape

In [ ]:
df_primates.head()

In [ ]:
df_primates.research_effort.hist()

In [ ]:
df_primates.brain.hist()

In [ ]:
df_primates['research_effort_log']=np.log(df_primates.research_effort)
df_primates['brain_log']=np.log(df_primates.brain)

df_primates.head()

In [ ]:
df_primates.shape


In [ ]:
df_primates=df_primates[~pd.isnull(df_primates.brain_log)]
df_primates.shape

In [ ]:
df_primates.social_learning.hist()

In [ ]:
df_primates.social_learning.describe()

In [ ]:
with pm.Model() as m_social_learning:
    a = pm.Normal("a", 0.0, 1.0)
    b_brain = pm.Normal("b_brain", 0.0, 0.5)

    brain_log = pm.Data("brain_log", df_primates.brain_log)
    lam = pm.math.exp(a+ b_brain * brain_log)

    social_learning = pm.Poisson("social_learning", lam, observed=df_primates.social_learning)
    trace_m_social_learning = pm.sample(random_seed=RANDOM_SEED)

In [ ]:
prior_pred.prior_predictive

In [ ]:
with m_social_learning:
    prior_pred = pm.sample_prior_predictive()
    az.plot_dist(prior_pred.prior_predictive["social_learning_observed"].stack(samples=("chain", "draw")), kind="hist")

In [ ]:
az.summary(trace_m_social_learning)

One unit increase of brain on log scale increase social learning np.exp(2.115) (about 8.29) units

In [ ]:
az.plot_trace(trace_m_social_learning)

In [ ]:
df_primates=df_primates[~pd.isnull(df_primates.research_effort_log)]
df_primates.shape

In [ ]:
with pm.Model() as m_social_learning2:
    a = pm.Normal("a", 0.0, 1.0)
    b_brain = pm.Normal("b_brain", 0.0, 0.5)
    b_effort = pm.Normal("b_effort", 0.0, 0.5)

    brain_log = pm.Data("brain_log", df_primates.brain_log)
    effort_log = pm.Data("effort_log", df_primates.research_effort_log)
    lam = pm.math.exp(a+ b_brain * brain_log+ b_effort*effort_log)

    social_learning = pm.Poisson("social_learning", lam, observed=df_primates.social_learning)
    trace_m_social_learning2 = pm.sample(random_seed=RANDOM_SEED)

In [ ]:
np.exp(1.584)

In [ ]:
az.summary(trace_m_social_learning2)

for effort coefficient now means that each 1 log unit of increase in effort it increases np.exp(1.584) (4.8 times) social learning count on average

 (c) Draw a DAG to represent how you think the variables social_learning, brain, and research_effort interact. Justify the DAG with the measured associations in the two models above (and any other models you used).

DAG could be Brain > Social learning, Brain > Research effort > Social Learning. Because more bigger brains mean more research effort. 

## Two ways of modelling multinomial data 


## 🧠 Comparison Summary

| **Feature**                          | **Multinomial (Categorical GLM)**       | **Poisson Transformation**             |
|--------------------------------------|------------------------------------------|----------------------------------------|
| **Distribution**                     | Categorical / Multinomial                | Multiple independent Poissons          |
| **Link function**                    | Softmax (multinomial logit)              | Log                                    |
| **Outcome data type**                | One outcome per row (id-level)           | Count per category (grouped data)      |
| **Handles probabilities**            | Directly                                 | Indirectly (via normalized rates)      |
| **Need to model \(K - 1\) categories** | Yes (requires a pivot)                  | No (model all \(K\) rates separately)  |
| **Conditioning on total count**      | No (built-in to likelihood)              | Yes (to recover multinomial behavior)  |
| **Interpreting coefficients**        | Tricky (log-odds relative to pivot)      | Log-rate scale (more interpretable)    |

---

## 📌 When to Use Which?

### ✅ Use **Multinomial GLM** when:
- You want probabilities directly.
- You have individual-level outcome data.
- You’re modeling **choice behavior**.

### ✅ Use **Poisson transformation** when:
- You have **grouped counts** per category.
- You prefer modeling **log-rates**.
- You want **simpler computation** or access to **Poisson GLM tools**.

---

## 🔮 Why do both give the same result?

Because the **Poisson transformation** is mathematically equivalent to the **multinomial**, *conditioned on the total count*.

You model the **unconditional rates** with Poisson and then **normalize** to get conditional probabilities — that’s what the multinomial does internally.

The math:

$$
\Pr(y_1, \ldots, y_k \mid n) = \frac{\prod_{i=1}^k \mathrm{Poisson}(y_i \mid \lambda_i)}{\mathrm{Poisson}\left(n \mid \sum_{i=1}^k \lambda_i\right)}
$$


So you can **freely switch** between the two approaches depending on your modeling goals and data structure.


## 🔗 Link Functions in Common GLMs

| **Model Type**           | **Distribution** | **Link Function**                                 | **Why This Link?**                                |
|--------------------------|------------------|---------------------------------------------------|---------------------------------------------------|
| Binomial (Bernoulli)     | Binomial         | $$ \text{logit}(p) = \log\left(\frac{p}{1 - p}\right) $$ | Ensures probability $$ p \in (0,1) $$             |
| Poisson regression       | Poisson          | $$ \log(\lambda) $$                                | Ensures rate $$ \lambda > 0 $$                    |
| Multinomial regression   | Multinomial      | $$ \log\left(\frac{p_k}{p_K}\right) $$ (Softmax)   | Models relative log-probabilities                 |

---

### ✅ 1. Binomial Regression

Used when outcomes are binary (e.g., success/failure).

- **Distribution**:  
  $$
  y \sim \text{Binomial}(n, p)
  $$

- **Link Function**:  
  $$
  \text{logit}(p) = \log\left(\frac{p}{1 - p}\right) = \alpha + \beta x
  $$

- **Why?**  
  Logit turns probabilities into real numbers, which we can model with a linear equation.

---

### ✅ 2. Poisson Regression

Used for **count data** (number of events).

- **Distribution**:  
  $$
  y \sim \text{Poisson}(\lambda)
  $$

- **Link Function**:  
  $$
  \log(\lambda) = \alpha + \beta x
  $$

- **Why?**  
  The log ensures that the expected rate $$ \lambda $$ is always positive.

---

### ✅ 3. Multinomial Regression

Used when there are **multiple outcome categories** (e.g., red/blue/green).

- **Distribution**:  
  $$
  y \sim \text{Multinomial}(p_1, ..., p_K)
  $$

- **Link Function** (Softmax / multinomial logit):  
  $$
  \log\left(\frac{p_k}{p_K}\right) = \alpha_k + \beta_k x \quad \text{for } k = 1, \dots, K-1
  $$

- **Why?**  
  The softmax ensures that all category probabilities are positive and sum to 1.

---

### 🔄 Summary Table

| **GLM Type**     | **Outcome**        | **Link Function**                                  | **Ensures…**                   |
|------------------|--------------------|----------------------------------------------------|--------------------------------|
| Binomial         | Binary (0/1)       | $$ \log\left(\frac{p}{1 - p}\right) $$             | $$ p \in (0,1) $$              |
| Poisson          | Counts (0, 1, …)   | $$ \log(\lambda) $$                                | $$ \lambda > 0 $$              |
| Multinomial      | Categories         | $$ \log\left(\frac{p_k}{p_K}\right) $$ (Softmax)   | $$ \sum p_k = 1 $$             |
